# Segmenting and Clustering Neighborhoods in Toronto

### Section 1: Retrieving Data

In [1]:
#Importing library
import pandas as pd
import numpy as np

In [2]:
#Importing Library
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print("Library Imported!")

Library Imported!


In [3]:
#Entering URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

# Retrieve all of the anchor tags
tags = soup('tr')
list_tags = list()

#Creating a list for each column
postal_code = list()
borough = list()
neighbourhood = list()

#Creating a list of strings from tags
for tag in tags:
    # Look at the parts of a tag
    list_tags.append(str(tag))

#Finding the data for each column from tags    
for line in list_tags:
    #Finding postal code data
    x = re.findall('M[0-9].',line)
    #Finding borough data
    y = re.findall('<tr>\n<td>M[0-9].\n</td>\n<td>(.*)', line)
    #Finding neighbourhood data
    z = re.findall('<tr>\n<td>M[0-9].\n</td>\n<td>.*\n</td>\n<td>(.*)',line)
    
    #Creating a list of postal_code column
    for a in x:
        postal_code.append(a)
    #Creating a list of borough column
    for b in y:
        borough.append(b)
    #Creating a list of neighbourhood column
    for c in z:
        neighbourhood.append(c)

In [4]:
#Creating a dataframe
df_toronto = pd.DataFrame()

df_toronto["postal_code"] = postal_code
df_toronto["borough"] = borough
df_toronto["neighbourhood"] = neighbourhood

#Checking the dataframe
df_toronto.head()

,postal_code,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#Assigning data from 'borough' column for rows in 'neighbourhood' that have 'Not assigned' data
df_toronto['neighbourhood'].replace('Not assigned', np.nan, inplace=True)
df_toronto.ffill(axis=1, inplace=True)

#Replacing 'Not assigned' borough with NaN
df_toronto['borough'].replace('Not assigned', np.nan, inplace=True)

#Deleting rows with 'Not assigned borough'
df_toronto.dropna(subset=['borough','neighbourhood'], axis=0, inplace=True)

#Resetting index
df_toronto.reset_index(inplace=True, drop=True)

#Checking the updated dataframe
df_toronto.tail()

,postal_code,borough,neighbourhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
#Printing the number of rows in the dataframe
df_toronto.shape

(103, 3)

### Section 2: Retrieving Latitude and Longitude Data
Because I was unable to retrieve data using Geocoder package, I used csv file instead to retrieve latitude and longitude data for each postal code in df_toronto dataframe.

In [7]:
#Adding latitude and longitude data using csv file

#Import csv file
geo_data = pd.read_csv('http://cocl.us/Geospatial_data')

#Merging df_toronto with geo_data to retrieve latitude and longitude data
df_toronto = df_toronto.merge(geo_data, how = 'left', left_on = 'postal_code', right_on = 'Postal Code')
df_toronto.head()


,postal_code,borough,neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [8]:
#Printing the number of rows in the data frame after being merged
df_toronto.shape

(103, 6)

In [9]:
#Deleting 'Postal Code column'
df_toronto.drop(['Postal Code'], axis=1, inplace=True)
df_toronto.head()

,postal_code,borough,neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Section 3: Exploring and Clustering Neighbourhoods in Toronto, Canada

In [10]:
#Importing folium library
import folium

#Latitude and longitude of Toronto
lat_toronto = 43.6532
long_toronto = -79.3832

#Creating map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat_toronto, long_toronto], zoom_start=10)

#Adding markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['borough'], df_toronto['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
#Filtering data containing 'Toronto' in borough column
borough_toronto = df_toronto[df_toronto['borough'].str.contains('Toronto')]

#Resetting index of borough_toronto
borough_toronto.reset_index(inplace=True, drop=True)

print(borough_toronto)

   postal_code           borough  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M4E      East Toronto   
5          M5E  Downtown Toronto   
6          M5G  Downtown Toronto   
7          M6G  Downtown Toronto   
8          M5H  Downtown Toronto   
9          M6H      West Toronto   
10         M5J  Downtown Toronto   
11         M6J      West Toronto   
12         M4K      East Toronto   
13         M5K  Downtown Toronto   
14         M6K      West Toronto   
15         M4L      East Toronto   
16         M5L  Downtown Toronto   
17         M4M      East Toronto   
18         M4N   Central Toronto   
19         M5N   Central Toronto   
20         M4P   Central Toronto   
21         M5P   Central Toronto   
22         M6P      West Toronto   
23         M4R   Central Toronto   
24         M5R   Central Toronto   
25         M6R      West Toronto   
26         M4S   Central Tor

In [41]:
borough_toronto.shape

(39, 5)

In [12]:
#Creating a map of boroughs containing 'Toronto' using latitude and longitude values
map_borough_toronto = folium.Map(location=[lat_toronto, long_toronto], zoom_start=11)

for lat, lng, borough, neighbourhood in zip(borough_toronto['Latitude'], borough_toronto['Longitude'], borough_toronto['borough'], borough_toronto['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough_toronto)  
    
map_borough_toronto

#### Using Foursquare API to Explore and Segment the Neighbourhoods

In [18]:
#Importing packages
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Defining Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'VPTPFDO1SIUWL1OAQJD522NH0KZK4GLUEFQTLFJABW2T2EG1'
CLIENT_SECRET = 'L1BDOLP5UQUT2NVLQQ53A550YYE2E3N0ZUGFP0OAEN3JLWWX'
VERSION = '20180605' #Foursquare API version
LIMIT = 100 #A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VPTPFDO1SIUWL1OAQJD522NH0KZK4GLUEFQTLFJABW2T2EG1
CLIENT_SECRET:L1BDOLP5UQUT2NVLQQ53A550YYE2E3N0ZUGFP0OAEN3JLWWX


Creating a function to explore the neighbourhoods using Foursquare API

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        #Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Exploring each neighbourhood using getNearbyVenues function

In [21]:
toronto_venues = getNearbyVenues(names=borough_toronto['neighbourhood'],
                                   latitudes=borough_toronto['Latitude'],
                                   longitudes=borough_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airp

Checking the size of toronto_venues dataframe

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(1600, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [24]:
#Checking how many venues were returned for each neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [25]:
#Checking how many unique categories from returned values
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


#### Analyzing each neighbourhood

In [26]:
#One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#Adding neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

#Moving neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
